# DP Mixup Experiment
Demonstrates differentially private mixup training and outputs standardized JSON results.

In [ ]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from opacus import PrivacyEngine
import numpy as np
from experiment_utils import ExperimentResult, save_result
import gdrive_utils

def mixup_data(x, y, alpha=0.4):
    lam = np.random.beta(alpha, alpha)
    index = torch.randperm(x.size(0))
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


In [ ]:
# prepare data/model and run a single training step for demo
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', download=True, transform=transforms.ToTensor()),
    batch_size=64, shuffle=True
)
model = nn.Sequential(nn.Flatten(), nn.Linear(28*28, 10))
optimizer = optim.SGD(model.parameters(), lr=0.1)
privacy_engine = PrivacyEngine()
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    epochs=1,
    target_epsilon=1.0,
    target_delta=1e-5,
    max_grad_norm=1.0,
)

criterion = nn.CrossEntropyLoss()
x, y = next(iter(train_loader))
mixed_x, y_a, y_b, lam = mixup_data(x, y)
optimizer.zero_grad()
pred = model(mixed_x)
loss = lam * criterion(pred, y_a) + (1-lam) * criterion(pred, y_b)
loss.backward()
optimizer.step()

epsilon = privacy_engine.get_epsilon(1e-5)
result = ExperimentResult(
    experiment_name='dp_mixup_demo',
    dataset='MNIST',
    model='fc',
    params={'batch_size':64, 'alpha':0.4},
    metrics={'loss':float(loss.item())},
    epsilon=epsilon,
    delta=1e-5
)
output_path = save_result(result, 'outputs/dp_mixup.json')
gdrive_utils.upload_file_to_gdrive(str(output_path), 'GDRIVE_FOLDER_ID')
result.to_json()
